In [1]:
import pandas as pd
import MySQLdb
import numpy as np
import time
import sys
import datetime 


j = 0
d = 7  #Day
all_p = 0
min_p = 0
all_power = []
sum_power = []
sum_p = []
k= 0

# Print iterations progress
# ราย15 วินาทีเป็น 15นาที แสดงว่ามีข้อมูล 96 ค่าข้อมูลต่อหนึ่งวัน ดังนั้นถ้าต้องการข้อมูลในรายชม.ในหนึ่งวัน จะต้องได้ 24ชั่วโมง = 24 ค่าข้อมูล 
time_a = pd.date_range('2016-09-01', periods=24*d, freq='60T')
time_b = pd.date_range('2016-09-01 00:59:59', periods=24*d, freq='60T')

conn= MySQLdb.connect("localhost","conn","hems","power")
c = conn.cursor()

####### set databases to empty #########
query_empty = """truncate table watt.watt_month""" 
c.execute(query_empty)
print "watt.watt_month are Empty!!! \n"

####### set databases_monthly_fee to empty #########
query_empty = """truncate table watt.unit_per_day""" 
c.execute(query_empty)
print "watt.unit_per_day are Empty!!! \n"

while (j <= (24*d)-1) :
    query = "SELECT Timestamp,s11 FROM power.enegate_bkk WHERE Timestamp >= '%s'   AND TimeStamp <= '%s'; "% (time_a[j],time_b[j])
    df = pd.read_sql(query, conn)
    df = df[df.s11 != "----"]
    df = df.reset_index(drop=True)
    df['s11'] = df['s11'].astype(float)
    
    if len(df) == 0 :
        min_p = 0
    else :
        for i in range(len(df)):
            min_p += df.s11[i]        
        min_p = min_p/len(df)  ##ข้อมูลพลังงานราย ชม.
    all_p += min_p
    print time_b[j] , "-->" ,min_p/1000 ," KW/Hr." #,all_p/1000 ," KW/Hr."
    
    c.execute("INSERT INTO watt.watt_month (Timestamp, watt) VALUES ( %s, %s)",(time_b[j],min_p))
    conn.commit()


### คำนวณพลังงานแต่ละวัน โดยการรับข้อมูลมาทั้งหมด ### 
    all_power.append(min_p/1000)
    df1 = pd.DataFrame(all_power)
    while ( k < (len(df1)/24) ) : 
        print """\n. . . . . . . . . . . . . . . . . . . .END DAY  . . . . . . . . . . . .. . . . . . . . . . . . . . .\n"""
#         print (k*24),"to",(k+1)*24
        sum_p =  df1[(k*24):((k+1)*24)].sum()
        sum_power.append(sum_p)
        df2 = pd.DataFrame(sum_power)  
        df2.columns = ['kW/day']
        print df2
        print "Power consuming between" ,time_a[k*24], "to" ,time_b[(k+1)*24-1], "equal to",df2.iloc[k]['kW/day']," kW." 
        print """///////////////////////////////////////////////////////////////////////////////////////////////////////////////\n"""
        c.execute("INSERT INTO watt.unit_per_day (Timestamp, unit) VALUES ( %s, %s)",(time_b[(k+1)*24-1],df2.iloc[k]['kW/day']))
        conn.commit()
        k +=1 
        
    min_p = 0
    j += 1
    
print(".DONE!")
print """///////////////////////////////////////////////////////////////////////////////////////////////////////////////\n"""
print "Power consuming between" ,time_a[0], "to" ,time_b[j-1], "equal to"  ,all_p/1000 ," kW/Hr = UNIT"   ##พลังงานรวมทั้งหมด
 

watt.watt_month are Empty!!! 

watt.unit_per_day are Empty!!! 

2016-09-01 00:59:59 --> 1.64969491525  KW/Hr.
2016-09-01 01:59:59 --> 1.47050847458  KW/Hr.
2016-09-01 02:59:59 --> 1.34938983051  KW/Hr.
2016-09-01 03:59:59 --> 1.32406779661  KW/Hr.
2016-09-01 04:59:59 --> 1.26842553191  KW/Hr.
2016-09-01 05:59:59 --> 1.28349152542  KW/Hr.
2016-09-01 06:59:59 --> 1.42789830508  KW/Hr.
2016-09-01 07:59:59 --> 1.99138723404  KW/Hr.
2016-09-01 08:59:59 --> 0.92186440678  KW/Hr.
2016-09-01 09:59:59 --> 0.85393220339  KW/Hr.
2016-09-01 10:59:59 --> 0.729804255319  KW/Hr.
2016-09-01 11:59:59 --> 0.443694915254  KW/Hr.
2016-09-01 12:59:59 --> 0.389796610169  KW/Hr.
2016-09-01 13:59:59 --> 0.476542372881  KW/Hr.
2016-09-01 14:59:59 --> 0.464170212766  KW/Hr.
2016-09-01 15:59:59 --> 0.553423728814  KW/Hr.
2016-09-01 16:59:59 --> 0.56786440678  KW/Hr.
2016-09-01 17:59:59 --> 0.862677966102  KW/Hr.
2016-09-01 18:59:59 --> 1.81174468085  KW/Hr.
2016-09-01 19:59:59 --> 1.90027118644  KW/Hr.
2016-09-0

35443.326439013566